In [ ]:
from ultralytics import YOLO
import cv2

def run_webcam_tracking(detect_model_path: str, pose_model_path: str) -> None:
    # Load the YOLO models
    detect_model: YOLO = YOLO(detect_model_path).to("mps")
    pose_model: YOLO = YOLO(pose_model_path).to("mps")
    
    # Open the webcam
    cap: cv2.VideoCapture = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open the webcam.")
        return

    # Set webcam resolution to 4K (3840x2160)
    cap.set(cv2.CAP_PROP_FRAME_WIDTH, 3840)
    cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)
    
    # Print the current resolution to verify
    width: float = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
    height: float = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
    print(f"Webcam resolution set to: {width} x {height} 📸")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Run YOLO object detection with ByteTrack enabled
        detect_results = detect_model.track(frame,  tracker="bytetrack.yaml")
        detect_annotated_frame = detect_results[0].plot()
        
        # Run YOLO pose estimation
        pose_results = pose_model(frame)
        pose_annotated_frame = pose_results[0].plot()
        
        # Merge both detections by overlaying them
        combined_frame = cv2.addWeighted(detect_annotated_frame, 0.5, pose_annotated_frame, 0.5, 0)
        
        # Display the combined frame
        cv2.imshow("YOLO Detection + Pose Estimation", combined_frame)
        
        # Exit the loop if 'q' is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the webcam and close the window
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    run_webcam_tracking("runs/detect/train-OD11n-v8/weights/best.pt", "../ultralytics_models/pose/yolo11n-pose.pt")
